## As observações de sentimentos podem ser usado para analisar as opiniões dos produtos

- aplicações de comentarios do twt
- aplicações de comentarios em fotos do instagram
- ou até mesmo no facebook

## Nesse caso o contexto é muito importante

### Para a rede ter multclasse usamos o ativador softmax

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dropout, Dense, LSTM, SpatialDropout1D, Embedding
from keras.utils import np_utils
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


In [2]:
Tweets = pd.read_csv('../dados/voos/Tweets.csv')
Tweets.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [3]:
Tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14640 entries, 0 to 14639
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   tweet_id                      14640 non-null  int64  
 1   airline_sentiment             14640 non-null  object 
 2   airline_sentiment_confidence  14640 non-null  float64
 3   negativereason                9178 non-null   object 
 4   negativereason_confidence     10522 non-null  float64
 5   airline                       14640 non-null  object 
 6   airline_sentiment_gold        40 non-null     object 
 7   name                          14640 non-null  object 
 8   negativereason_gold           32 non-null     object 
 9   retweet_count                 14640 non-null  int64  
 10  text                          14640 non-null  object 
 11  tweet_coord                   1019 non-null   object 
 12  tweet_created                 14640 non-null  object 
 13  t

In [4]:
Tweets.groupby(['airline_sentiment']).size() # faz uma contagem dos dados

airline_sentiment
negative    9178
neutral     3099
positive    2363
dtype: int64

In [5]:
Tweets.groupby(['airline_sentiment_gold']).size()

airline_sentiment_gold
negative    32
neutral      3
positive     5
dtype: int64

In [6]:
Tweets = Tweets[Tweets['airline_sentiment_confidence']>0.8] # essa coluna é a certeza do sentimento
Tweets.groupby(['airline_sentiment']).size()

airline_sentiment
negative    7392
neutral     1550
positive    1517
dtype: int64

#### Criação dos ‘tokens’

In [7]:
token = Tokenizer(num_words=100)
token.fit_on_texts(Tweets['text'].values)

#### usando o modelo para x

In [8]:
X = token.texts_to_sequences(Tweets['text'].values)
X = pad_sequences(X, padding="post", maxlen=100)

In [9]:
print(X)

[[97 62  0 ...  0  0  0]
 [97 99  1 ...  0  0  0]
 [97  9 99 ...  0  0  0]
 ...
 [13 98 93 ...  0  0  0]
 [13 89  1 ...  0  0  0]
 [13  6 23 ...  0  0  0]]


In [10]:
labelencoder = LabelEncoder()
y = labelencoder.fit_transform(Tweets['airline_sentiment']) # aqui transformamos as respostas em numeros
print(y)

[1 0 0 ... 0 1 0]


In [11]:
y = np_utils.to_categorical(y) # aqui temos uma matrix informando qual é a categoria com 1
print(y)

[[0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 ...
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]]


In [12]:
X_treino, X_teste, y_treino, y_teste, = train_test_split(X, y, test_size=0.3) # separa quem é quem

### iniciando o modelo

In [13]:
X.shape[1]

100

In [14]:
modelo = Sequential() # <- modelo de deepLearn LTMS

input_dim = entradas de neuronios
input_length = tamanho da sequencia de entrada dos dados

In [15]:
modelo.add(Embedding(input_dim=len(token.word_index), output_dim=128, input_length=X.shape[1]))

In [16]:
modelo.add(SpatialDropout1D(0.2))

In [17]:
modelo.add(LSTM(units=196, dropout=0.2, recurrent_dropout=0, activation="tanh", recurrent_activation="sigmoid",unroll=False, use_bias=True))

In [18]:
modelo.add(Dense(units=3, activation="softmax"))

In [19]:
modelo.compile(loss="categorical_crossentropy", optimizer="adam", metrics=['accuracy'])
modelo.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 128)          1638656   
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 100, 128)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 591       
Total params: 1,894,047
Trainable params: 1,894,047
Non-trainable params: 0
_________________________________________________________________


In [ ]:
modelo.fit(X_treino, y_treino, epochs=30, batch_size=30, verbose=True, validation_data=(X_teste, y_teste))

C:\Users\jefferson\.conda\envs\gpu\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 7321 samples, validate on 3138 samples
Epoch 1/30


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\jefferson\.conda\envs\gpu\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\jefferson\AppData\Local\Temp\ipykernel_11456\1962250636.py", line 1, in <module>
    modelo.fit(X_treino, y_treino, epochs=30, batch_size=30, verbose=True, validation_data=(X_teste, y_teste))
  File "C:\Users\jefferson\.conda\envs\gpu\lib\site-packages\keras\engine\training.py", line 1239, in fit
    validation_freq=validation_freq)
  File "C:\Users\jefferson\.conda\envs\gpu\lib\site-packages\keras\engine\training_arrays.py", line 196, in fit_loop
    outs = fit_function(ins_batch)
  File "C:\Users\jefferson\.conda\envs\gpu\lib\site-packages\tensorflow_core\python\keras\backend.py", line 3740, in __call__
    outputs = self._graph_fn(*converted_inputs)
  File "C:\Users\jefferson\.conda\envs\gpu\lib\site-packages\tensorflow_core\python\eager\function.py", line 1081,

In [ ]:
loss, accuracy = modelo.evaluate(X_teste, y_teste)
print(f"loss: {loss}\n accuracy: {accuracy}")

In [ ]:
prev = modelo.predict(X_teste)
print(prev)